# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 9:02AM,243741,12,HH-36316,Hush Hush,Released
1,Aug 9 2022 9:02AM,243741,12,HH-36317,Hush Hush,Released
2,Aug 9 2022 9:02AM,243741,12,HH-36318,Hush Hush,Released
3,Aug 9 2022 9:02AM,243741,12,HH-36319,Hush Hush,Released
4,Aug 9 2022 9:02AM,243741,12,HH-36320,Hush Hush,Released
5,Aug 9 2022 9:02AM,243741,12,HH-36322,Hush Hush,Released
6,Aug 9 2022 9:02AM,243741,12,HH-36323,Hush Hush,Released
7,Aug 9 2022 9:02AM,243741,12,HH-36324,Hush Hush,Released
8,Aug 9 2022 9:02AM,243741,12,HH-36326,Hush Hush,Released
9,Aug 9 2022 9:02AM,243741,12,HH-36327,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,243735,Released,1
31,243736,Released,1
32,243738,Released,1
33,243740,Released,1
34,243741,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243735,NaN,1.0
243736,NaN,1.0
243738,NaN,1.0
243740,NaN,1.0
243741,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243677,0.0,1.0
243682,0.0,1.0
243684,16.0,1.0
243687,14.0,1.0
243689,19.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243677,0,1
243682,0,1
243684,16,1
243687,14,1
243689,19,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243677,0,1
1,243682,0,1
2,243684,16,1
3,243687,14,1
4,243689,19,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243677,,1
1,243682,,1
2,243684,16,1
3,243687,14,1
4,243689,19,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 9:02AM,243741,12,Hush Hush
12,Aug 9 2022 9:00AM,243740,19,Else Nutrition
13,Aug 9 2022 8:41AM,243738,10,Bio-PRF
14,Aug 9 2022 8:35AM,243736,10,Eywa Pharma Inc.
15,Aug 9 2022 8:31AM,243733,10,ISDIN Corporation
46,Aug 9 2022 8:30AM,243735,10,ISDIN Corporation
47,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation
53,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation
68,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation
86,Aug 9 2022 8:29AM,243731,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 9 2022 9:02AM,243741,12,Hush Hush,,12
1,Aug 9 2022 9:00AM,243740,19,Else Nutrition,,1
2,Aug 9 2022 8:41AM,243738,10,Bio-PRF,,1
3,Aug 9 2022 8:35AM,243736,10,Eywa Pharma Inc.,,1
4,Aug 9 2022 8:31AM,243733,10,ISDIN Corporation,,31
5,Aug 9 2022 8:30AM,243735,10,ISDIN Corporation,,1
6,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,,6
7,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,,15
8,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,,18
9,Aug 9 2022 8:29AM,243731,10,ISDIN Corporation,,48


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 9:02AM,243741,12,Hush Hush,12,
1,Aug 9 2022 9:00AM,243740,19,Else Nutrition,1,
2,Aug 9 2022 8:41AM,243738,10,Bio-PRF,1,
3,Aug 9 2022 8:35AM,243736,10,Eywa Pharma Inc.,1,
4,Aug 9 2022 8:31AM,243733,10,ISDIN Corporation,31,
5,Aug 9 2022 8:30AM,243735,10,ISDIN Corporation,1,
6,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6,
7,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15,
8,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18,
9,Aug 9 2022 8:29AM,243731,10,ISDIN Corporation,48,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 9:02AM,243741,12,Hush Hush,12,
1,Aug 9 2022 9:00AM,243740,19,Else Nutrition,1,
2,Aug 9 2022 8:41AM,243738,10,Bio-PRF,1,
3,Aug 9 2022 8:35AM,243736,10,Eywa Pharma Inc.,1,
4,Aug 9 2022 8:31AM,243733,10,ISDIN Corporation,31,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 9:02AM,243741,12,Hush Hush,12.0,NaN
1,Aug 9 2022 9:00AM,243740,19,Else Nutrition,1.0,NaN
2,Aug 9 2022 8:41AM,243738,10,Bio-PRF,1.0,NaN
3,Aug 9 2022 8:35AM,243736,10,Eywa Pharma Inc.,1.0,NaN
4,Aug 9 2022 8:31AM,243733,10,ISDIN Corporation,31.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 9:02AM,243741,12,Hush Hush,12.0,0.0
1,Aug 9 2022 9:00AM,243740,19,Else Nutrition,1.0,0.0
2,Aug 9 2022 8:41AM,243738,10,Bio-PRF,1.0,0.0
3,Aug 9 2022 8:35AM,243736,10,Eywa Pharma Inc.,1.0,0.0
4,Aug 9 2022 8:31AM,243733,10,ISDIN Corporation,31.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3655777,142.0,0.0
102,974861,4.0,0.0
12,243741,12.0,0.0
16,243703,0.0,1.0
19,974888,2.0,5.0
20,974768,32.0,49.0
21,731163,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3655777,142.0,0.0
1,102,974861,4.0,0.0
2,12,243741,12.0,0.0
3,16,243703,0.0,1.0
4,19,974888,2.0,5.0
5,20,974768,32.0,49.0
6,21,731163,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,142.0,0.0
1,102,4.0,0.0
2,12,12.0,0.0
3,16,0.0,1.0
4,19,2.0,5.0
5,20,32.0,49.0
6,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,142.0
1,102,Released,4.0
2,12,Released,12.0
3,16,Released,0.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,19,20,21
Status,,,,,,,
Executing,0.0,0.0,0.0,1.0,5.0,49.0,0.0
Released,142.0,4.0,12.0,0.0,2.0,32.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,19,20,21
0,Executing,0.0,0.0,0.0,1.0,5.0,49.0,0.0
1,Released,142.0,4.0,12.0,0.0,2.0,32.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,19,20,21
0,Executing,0.0,0.0,0.0,1.0,5.0,49.0,0.0
1,Released,142.0,4.0,12.0,0.0,2.0,32.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()